In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
res_d <- '/scratch/groups/mrivas/projects/PRS/private_output/202009_batch'


In [3]:
eval_1_df <- fread(file.path(res_d, 'snpnet.eval.1_fit_w_val.tsv')) %>% rename('GBE_ID'='#GBE_ID')
eval_2_df <- fread(file.path(res_d, 'snpnet.eval.2_refit.tsv')) %>% rename('GBE_ID'='#GBE_ID')

In [4]:
bind_rows(
    eval_1_df %>% mutate(run_idx = '1_fit_w_val'),
    eval_2_df %>% mutate(run_idx = '2_refit')
) %>%
filter(split == 'test', n_variables>0) %>%
rename('metric'='geno_covar') %>%
select(GBE_ID, family, run_idx, metric) %>%
unique() %>% spread(run_idx, metric) -> df

In [5]:
plots <- list()
for(f in c('binomial', 'gaussian')) {
    df %>% drop_na(`1_fit_w_val`, `2_refit`) %>%
    filter(family == f) %>%
    ggplot(aes(x = `1_fit_w_val`, y = `2_refit`)) + 
    geom_abline(slope=1, intercept=0, color='red') + 
    geom_point(alpha=.3) + theme_bw()+
    labs(
        title=ifelse(
            f == 'binomial', 
            'Binary phenotypes (family: Binomial)', 
            'Quantitative phenotypes (family: Gaussian)'
        ),
        x = 'Model performance trainined on training (70%) and validation (10%) sets',
        y = 'Model performance trainined on the combined (70% + 10%) set'
    ) +
    geom_vline(xintercept = ifelse(f == 'binomial', 0.5, 0), color='gray') + 
    geom_hline(yintercept = ifelse(f == 'binomial', 0.5, 0), color='gray') -> plots[[f]]
}


In [6]:
ggsave('8_plot_eval.png', gridExtra::arrangeGrob(plots[['binomial']], plots[['gaussian']], ncol=2), width=12, height=6)

In [7]:
summary(lm(`2_refit` ~ 0 + `1_fit_w_val`, df %>% drop_na(`1_fit_w_val`, `2_refit`) %>% filter(family == 'binomial')))$coefficients


,Estimate,Std. Error,t value,Pr(>|t|)
`1_fit_w_val`,1.002266,0.0003903395,2567.678,0


In [8]:
summary(lm(`2_refit` ~ 0 + `1_fit_w_val`, df %>% drop_na(`1_fit_w_val`, `2_refit`) %>% filter(family == 'gaussian')))$coefficients


,Estimate,Std. Error,t value,Pr(>|t|)
`1_fit_w_val`,1.006895,0.0004027447,2500.083,0
